##### Load Data

In [1]:
from bs4                     import BeautifulSoup
import matplotlib.pyplot     as plt
import matplotlib.image      as mpimg
import numpy                 as np
import skimage.transform
import dicom
import glob
import model
import scipy.misc
import os
import h5py
import json
import pylab

data_filepath = "/enc_data/eddata/pacemaker/"
ann_filepath = data_filepath + "ann/"
img_filepath = data_filepath + 'organized-data/'
png_filepath = data_filepath + 'png/'

SIZE_X = 726
SIZE_Y = 726
VGG_DIMS = (224, 224)

Using TensorFlow backend.


In [50]:
def load_data():
    def resize(image):
        """ Resizes the image into dimensions specified by dims and converts img to 3 channels """
        newimg = skimage.transform.resize(image, VGG_DIMS)

        # Convert from 1 channel to 3 channels
        newimg_3d = np.empty(VGG_DIMS + (3,))
        for i in range(3):
            newimg_3d[:,:,i] = newimg

        return newimg_3d

    def load_images(path, number):
        """ Path: Path to .png images
            Number: Number of images to run
            Dims: (height, width) of the final images that will be fed into the model """

        return np.array([ resize(plt.imread(path + image_path)) for image_path in os.listdir(path)[:number] ])
    
    pos_imgs = load_images(png_filepath + "pacemaker/max_size/", number=300)
    neg_imgs = load_images(png_filepath + "neg/", number=300)
    
    X = np.concatenate((pos_imgs, neg_imgs))
    Y = np.array([1] * pos_imgs.shape[0] + [0] * neg_imgs.shape[0])
    print X.shape, Y.shape
    return X, Y

In [51]:
X, Y = load_data()

(600, 224, 224, 3) (600,)


##### Model

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from sklearn.model_selection         import train_test_split
from keras.applications.vgg16        import VGG16
from keras.applications.inception_v3        import InceptionV3


from keras.preprocessing.image       import ImageDataGenerator
from keras.models                    import Sequential, Model, Input
from keras.layers                    import Dense, Flatten, Dropout
from keras.optimizers                import SGD, RMSprop, Adam
from keras.callbacks                 import EarlyStopping
from keras.utils                     import np_utils
from keras                           import backend as K

In [52]:
# Split into test and train/validation set
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.6, random_state=42, stratify=Y)

# Split into train and validation
X_test, X_val, y_test, y_val= train_test_split(X_test, y_test, train_size=0.5, random_state=42, stratify=y_test) 
print('Train shape: ', X_train.shape, y_train.shape)
print('Val shape: ', X_val.shape, y_val.shape)
print('Test shape: ', X_test.shape, np.array(y_test).shape)

('Train shape: ', (360, 224, 224, 3), (360,))
('Val shape: ', (120, 224, 224, 3), (120,))
('Test shape: ', (120, 224, 224, 3), (120,))


In [53]:
# Categorize the labels
num_classes = 2
y_train = np_utils.to_categorical(y_train, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
print("y_train, y_val, y_test: ", y_train.shape, y_val.shape, y_test.shape)

('y_train, y_val, y_test: ', (360, 2), (120, 2), (120, 2))


In [54]:
# Create the base pre-trained model
base_model = InceptionV3(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

x = base_model.output
x = Flatten()(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [55]:
model = Model(inputs=base_model.input, outputs=predictions)
# k = 1 # number of end layers to retrain
# layers = base_model.layers[:-k] if k != 0 else base_model.layers
# for layer in layers: 
#     layer.trainable = False
# #print(model.summary())

In [60]:
opt = SGD(lr=0.0001, momentum=0.9)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
model.fit(x=X_train, y=y_train, epochs=10)

Epoch 1/10
360/360 [==============================] - 9s 24ms/step - loss: 0.0058 - acc: 1.0000
Epoch 2/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0098 - acc: 0.9972
Epoch 3/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0186 - acc: 0.9944
Epoch 4/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0119 - acc: 1.0000
Epoch 5/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0042 - acc: 1.0000
Epoch 6/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0107 - acc: 0.9972
Epoch 7/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0067 - acc: 1.0000
Epoch 8/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0075 - acc: 1.0000
Epoch 9/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0111 - acc: 0.9972
Epoch 10/10
360/360 [==============================] - 2s 7ms/step - loss: 0.0042 - acc: 1.0000


In [61]:
test_loss, accuracy = model.evaluate(x=X_test, y=y_test, verbose=1)
print(accuracy)

120/120 [==============================] - 2s 21ms/step
0.966666666667
